In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import random

from ts_pred_helper import *
from model import *
from data import *

## Nonlinear model performance when using different number of neurons

In [ ]:
iterations = 3
model_names = ['LinearFFx4', 'FFx4', 'RNNx4', 'Transformerx4']

# data generation
init_seq = np.array([0.1, 0.2, 0.3])
weights = np.array([0.1, 0.2, 0.7, 0.1])
noise_func = gaussian_noise
data = get_data(data_type='nonlinear_ar', init_seq=init_seq, weights=weights, noise_func=noise_func, normalize=False)

# widths
x_ticks = np.linspace(2, 8, 7)
widths = [int(2**x_tick) for x_tick in x_ticks]

def train_and_save(idx):
    results = {}
    for width in tqdm(widths):
        models = [LinearFF(1, 3, width), FF(1, 3, width), RNN(1, width, 4, 1), Transformer(1, width, 2, 4, 3)]
        results[width] = []
        for i in range(iterations):
            set_seed(i+1)
            tys, pys = train_eval1(models[idx], data=data, window_size=3, epochs=200)
            results[width].append((tys, pys))
    pickle_save(f'results/{model_names[idx]}_res.pkl', results)


def load_and_eval(idx, threhold):
    ff_res = pickle_load(f'results/{model_names[idx]}_res.pkl')
    x_ticks = []
    ys1 = []
    for key, res_list in ff_res.items():
        x_ticks.append(np.log2(key))
        cur_ys = []
        for value in res_list:
            tys, pys = value
            cur_y = rmse(pys, tys)
            if cur_y > threhold:
                cur_y = threhold
            cur_ys.append(cur_y)
        ys1.append(np.mean(cur_ys))
    return x_ticks, ys1